In [2]:
import tkinter as tk
from tkinter import messagebox

# Onsight Competition Score Application

## Goals
What should this application be able to do?  

**Basics**  
* Be able to enter the following per boulder:
    * Top (if yes how many attempts)
    * Zone (if yes how many attempts)
    * Attempts (total if neither zone nor top achieved)
* Be able to enter Tops/Zones/Attempts for each boulder in the round.
* Create leaderboard ranking based on entered information.

**Advanced**  
* Somehow link functionality to RO_app. Where the program takes information like the number of boulders per round, and list of competitors. (May need to overhaul the layout and present a more OOP style to the backend.)
* Active controls once base program is set up.

In [6]:
# OOP Scoring System
class Scores:
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops}T    {self.attempts} A"
    
    class Zone:
        def __init__(self, zones = 0, attempts = 0):
            self.zones = zones
            self.attempts = attempts

        def __str__(self):
            return f"{self.zones}Z    {self.attempts} A"

    class LowZone:
        def __init__(self, low_zones = 0, attempts = 0):
            self.low_zones = low_zones
            self.attempts = attempts

        def __str__(self):
            return f"{self.low_zones}LZ    {self.attempts} A"

    def __init__(self):
        self.tops = Scores.Top(0, 0)
        self.zones = Scores.Zone(0, 0)
        self.low_zones = Scores.LowZone(0, 0)

class Climber:
    def __init__(self, name, category):
        self.name = name
        self.scores = Scores()
        self.category = category

    def __str__(self):
        return f"{self.name}:\n{self.scores.tops}\n{self.scores.zones}\n{self.scores.low_zones}"
    
class Leaderboard():
    def __init__(self):
        self.climbers = []

    def add_climber(self, *climbers):
        """Adds a climber to the leaderboard"""
        self.climbers.extend(climbers)

    def display_leaderboard(self):
        for climber in self.climbers:
            print(climber)
    
    def rank_climbers(self):
        """Sorts climbers by their scores. Key currently matches the USA Climbing Isolation scoring system."""
        def sort_key(climber):
            return (-climber.scores.tops.tops, -climber.scores.zones.zones, -climber.scores.low_zones.low_zones, 
            climber.scores.tops.attempts, climber.scores.zones.attempts, climber.scores.low_zones.attempts)
        self.climbers.sort(key=sort_key)
    

def create_climber(name, category, tops, zones, low_zones = (0, 0)):
    """Creates a climber object with the given name, category, and scores. Low zones are optional and scores are taken as tuples."""
    climber = Climber(name, category)
    climber.scores.tops = Scores.Top(*tops)
    climber.scores.zones = Scores.Zone(*zones)
    climber.scores.low_zones = Scores.LowZone(*low_zones)
    return climber

In [13]:
import tkinter as tk
from tkinter import ttk, messagebox

class ScoringApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        self.leaderboard = Leaderboard()
        
        # Labels and Entry Fields
        ttk.Label(root, text="Climber Name:").grid(row=0, column=0, padx=5, pady=5)
        self.name_entry = ttk.Entry(root)
        self.name_entry.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(root, text="Category:").grid(row=1, column=0, padx=5, pady=5)
        self.category_entry = ttk.Entry(root)
        self.category_entry.grid(row=1, column=1, padx=5, pady=5)
        
        ttk.Label(root, text="Tops (count, attempts):").grid(row=2, column=0, padx=5, pady=5)
        self.tops_entry = ttk.Entry(root)
        self.tops_entry.grid(row=2, column=1, padx=5, pady=5)
        
        ttk.Label(root, text="Zones (count, attempts):").grid(row=3, column=0, padx=5, pady=5)
        self.zones_entry = ttk.Entry(root)
        self.zones_entry.grid(row=3, column=1, padx=5, pady=5)
        
        ttk.Label(root, text="Low Zones (count, attempts):").grid(row=4, column=0, padx=5, pady=5)
        self.low_zones_entry = ttk.Entry(root)
        self.low_zones_entry.grid(row=4, column=1, padx=5, pady=5)
        
        # Buttons
        ttk.Button(root, text="Add Climber", command=self.add_climber).grid(row= 5, column= 0, columnspan= 2, pady= 10)
        ttk.Button(root, text="Show Leaderboard", command=self.show_leaderboard).grid(row= 6, column= 0, columnspan= 2, pady= 10)
        ttk.Button(root, text="Clear Leaderboard", command=self.clear_leaderboard).grid(row= 8, column= 0, columnspan= 2, pady= 10)
        
        # Leaderboard Display
        self.leaderboard_text = tk.Text(root, height=10, width=50)
        self.leaderboard_text.grid(row=7, column=0, columnspan=2, pady=10)

    def clear_entries(self):
        """Clears all but category entry fields"""
        self.name_entry.delete(0, tk.END)
        self.tops_entry.delete(0, tk.END)
        self.zones_entry.delete(0, tk.END)
        self.low_zones_entry.delete(0, tk.END)

    def add_climber(self):
        """Adds a climber to the current leaderboard"""
        try:
            name = self.name_entry.get()
            category = self.category_entry.get()
            tops = tuple(map(int, self.tops_entry.get().split(',')))
            zones = tuple(map(int, self.zones_entry.get().split(',')))
            low_zones = tuple(map(int, self.low_zones_entry.get().split(','))) if self.low_zones_entry.get() else (0, 0)
            
            climber = create_climber(name, category, tops, zones, low_zones)
            self.leaderboard.add_climber(climber)
            messagebox.showinfo("Success", f"{name} added to the leaderboard!")
            self.clear_entries()
        except Exception as e:
            messagebox.showerror("Error", f"Invalid input: {e}")

    def show_leaderboard(self):
        """Displays the current leaderboard, with rankings"""
        self.leaderboard.rank_climbers()
        self.leaderboard_text.delete(1.0, tk.END)
        for climber in self.leaderboard.climbers:
            self.leaderboard_text.insert(tk.END, f"{climber}\n\n")

    def clear_leaderboard(self):
        """Clears the leaderboard values"""
        self.leaderboard.climbers = []
        self.leaderboard_text.delete(1.0, tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    app = ScoringApp(root)
    root.mainloop()
